In [1]:
import ast
from typing import List
from os import listdir
def get_streamings(path: str = 'C:/Users/niveatha/Desktop/Spotify/') -> List[dict]:
    
   
    
    files = ['C:/Users/niveatha/Desktop/Spotify/' + x for x in listdir(path)
             if x.split('.')[0][:-1] == 'StreamingHistory']
    
    all_streamings = []
    
    for file in files: 
        with open(file, 'r', encoding='UTF-8') as f:
            new_streamings = ast.literal_eval(f.read())
            all_streamings += [streaming for streaming 
                               in new_streamings]
    return all_streamings

In [2]:
import spotipy.util as util

username = 'nive sp analysis'
client_id ='cbaa1f2a892d4ed591c17c1544e39cfe'
client_secret = 'e707b64e517d42f592065ffb7552e916'
redirect_uri = 'http://localhost:7777/callback?code=AQDDriAvIyBBHjplQwr8bC88lTU3Jy_m1_LCNaLLzLKcVuhXYJOZ55WapK2dsFUCXEL9pxefEn_b3T9G1xRx8OBn2da5RkzSyPE9JwNwlDbXALd-ffBh9r18yycm0Si4IDg_CHrRsWh5tqRmY7n8XOWrYUbYTHBVexdOpadxK67-XbE6HuEaMMZqZLOxCKkCUfHNAqId8ZiU6Oqodw'
scope = 'user-read-recently-played'

token = util.prompt_for_user_token(username=username, 
                                   scope=scope, 
                                   client_id=client_id,   
                                   client_secret=client_secret,     
                                   redirect_uri=redirect_uri)

In [3]:
print(token)

BQDuUNOcwJXqbHOyAe7n1bK4WqhNwaJjGFHiQ1dIiuyIa2qrIj3Xq2d2K-_79gVwtXlWzNoZcz9TxCkiKae5IluUHh_V5RoxJgLEkSl2qj3AvSLhsoCpTGCcdjMCR-22dR58sJPqa32m046QjkINfWcYtzgbBBWYp66VE6Y


In [4]:
import requests
def get_id(track_name: str, token: str) -> str:
    headers={'Accept': 'application/json','Content-Type': 'application/json','Authorization': f'Bearer ' + token,}
    params=[('q', track_name),('type', 'track'),]
    try:
        response = requests.get('https://api.spotify.com/v1/search',headers = headers, params = params, timeout=5)
        json =response.json()
        first_result = json['tracks']['items'][0]
        track_id = first_result['id']
        return track_id
    except:
        return None

In [5]:
bed_id = get_id('In My Bed', token)
print(bed_id)


1FjD1jpm51dH5LzLvrDVPY


In [11]:
import spotipy.util as util
import spotipy
def get_features(track_id: str, token: str) -> dict:
    sp = spotipy.Spotify(auth=token)
    try:
        features = sp.audio_features([track_id])
        return features[0]
    except:
        return None

In [12]:
import spotipy.util as util


lucy_features = get_features(bed_id, token)
print(lucy_features)

{'danceability': 0.545, 'energy': 0.385, 'key': 9, 'loudness': -6.01, 'mode': 0, 'speechiness': 0.299, 'acousticness': 0.0714, 'instrumentalness': 0.0014, 'liveness': 0.0859, 'valence': 0.655, 'tempo': 218.365, 'type': 'audio_features', 'id': '1FjD1jpm51dH5LzLvrDVPY', 'uri': 'spotify:track:1FjD1jpm51dH5LzLvrDVPY', 'track_href': 'https://api.spotify.com/v1/tracks/1FjD1jpm51dH5LzLvrDVPY', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/1FjD1jpm51dH5LzLvrDVPY', 'duration_ms': 189981, 'time_signature': 4}


In [173]:
streamings = get_streamings()
unique_tracks = list(set([streaming['trackName'] 
                for streaming in streamings]))

all_features = {}
for track in unique_tracks:
    track_id = get_id(track, token)
    features = get_features(track_id, token)
    if features:
        all_features[track] = features
        
with_features = []
for track_name, features in all_features.items():
    with_features.append({'name': track_name, **features})

In [174]:
import pandas as pd
df = pd.DataFrame(with_features)
df.to_csv('streaming_history.csv')